In [0]:
from pyspark.sql.functions import desc, count, avg

In [0]:
%run ../source_to_bronze/utils

In [0]:
%run ../bronze_to_silver/employee_bronze_to_silver

Out[110]: <function __main__.camel_to_snake_case(df)>

Out[119]: <function __main__.camel_to_snake_case(df)>

In [0]:
employee_df = spark.read.format("delta").load('dbfs:/FileStore/assignments/questoin1/silver/employee_info/dim_employee')
display(employee_df)

employeeid,employeename,department,country,salary,age,load_date
1,James,D101,IN,9000,25,2024-04-16
2,Michel,D102,SA,8000,26,2024-04-16
3,James son,D101,IN,10000,35,2024-04-16
4,Robert,D103,MY,11000,34,2024-04-16
5,Scott,D104,MA,6000,36,2024-04-16
6,Gen,D105,JA,21345,24,2024-04-16
7,John,D102,MY,87654,40,2024-04-16
8,Maria,D105,SA,38144,38,2024-04-16
9,Soffy,D103,IN,23456,29,2024-04-16
10,Amy,D103,CN,21345,24,2024-04-16


EmployeeID,EmployeeName,Department,Country,Salary,Age
1,James,D101,IN,9000,25
2,Michel,D102,SA,8000,26
3,James son,D101,IN,10000,35
4,Robert,D103,MY,11000,34
5,Scott,D104,MA,6000,36
6,Gen,D105,JA,21345,24
7,John,D102,MY,87654,40
8,Maria,D105,SA,38144,38
9,Soffy,D103,IN,23456,29
10,Amy,D103,CN,21345,24


CountryCode,CountryName
CN,China
IN,India
SA,South Africa
JA,Japan
MY,Malaysia
MA,Morocco


DepartmentID,DepartmentName
D101,Sales
D102,Marketing
D103,Finance
D104,Support
D105,HR


employeeid,employeename,department,country,salary,age
1,James,D101,IN,9000,25
2,Michel,D102,SA,8000,26
3,James son,D101,IN,10000,35
4,Robert,D103,MY,11000,34
5,Scott,D104,MA,6000,36
6,Gen,D105,JA,21345,24
7,John,D102,MY,87654,40
8,Maria,D105,SA,38144,38
9,Soffy,D103,IN,23456,29
10,Amy,D103,CN,21345,24


departmentid,departmentname
D101,Sales
D102,Marketing
D103,Finance
D104,Support
D105,HR


countrycode,countryname
CN,China
IN,India
SA,South Africa
JA,Japan
MY,Malaysia
MA,Morocco


EmployeeID,EmployeeName,Department,Country,Salary,Age
1,James,D101,IN,9000,25
2,Michel,D102,SA,8000,26
3,James son,D101,IN,10000,35
4,Robert,D103,MY,11000,34
5,Scott,D104,MA,6000,36
6,Gen,D105,JA,21345,24
7,John,D102,MY,87654,40
8,Maria,D105,SA,38144,38
9,Soffy,D103,IN,23456,29
10,Amy,D103,CN,21345,24


DepartmentID,DepartmentName
D101,Sales
D102,Marketing
D103,Finance
D104,Support
D105,HR


countrycode,countryname,load_date
CN,China,2024-04-16
IN,India,2024-04-16
SA,South Africa,2024-04-16
JA,Japan,2024-04-16
MY,Malaysia,2024-04-16
MA,Morocco,2024-04-16


In [0]:
salary_of_department = employee_with_date_df.orderBy(desc('salary'))
display(salary_of_department.select('department', 'salary'))

department,salary
D102,87654
D105,38144
D103,23456
D105,21345
D103,21345
D103,11000
D101,10000
D101,9000
D102,8000
D104,6000


In [0]:
employee_count = employee_df.groupBy("department", "country").agg(count("employeeid").alias("employee_count"))
display(employee_count)

department,country,employee_count
D103,MY,1
D103,IN,1
D105,JA,1
D102,SA,1
D101,IN,2
D102,MY,1
D103,CN,1
D105,SA,1
D104,MA,1


In [0]:
department_join_df = employee_with_date_df.join(department_with_date_df, employee_with_date_df.department == department_with_date_df.departmentid, "inner")
country_join_df = department_join_df.join(country_with_date_df, department_join_df.country == country_with_date_df.countrycode, "inner")
department_with_country = country_join_df.select('departmentname', 'countryname')
display(department_with_country)

departmentname,countryname
Sales,India
Marketing,South Africa
Sales,India
Finance,Malaysia
Support,Morocco
HR,Japan
Marketing,Malaysia
HR,South Africa
Finance,India
Finance,China


In [0]:
avg_age_employee = employee_with_date_df.groupBy('department').agg(avg("age").alias('avg_age'))
display(avg_age_employee)

department,avg_age
D105,31.0
D104,36.0
D103,29.0
D101,30.0
D102,33.0


In [0]:
employee_with_date_df.write.format("parquet").mode("overwrite").option("replaceWhere", "load_date = '2024-04-16'").save("/FileStore/assignments/gold/employee/table_name")

In [0]:
test_df = spark.read.parquet('dbfs:/FileStore/assignments/gold/employee/table_name/')
display(test_df)

employeeid,employeename,department,country,salary,age,load_date
1,James,D101,IN,9000,25,2024-04-16
2,Michel,D102,SA,8000,26,2024-04-16
3,James son,D101,IN,10000,35,2024-04-16
4,Robert,D103,MY,11000,34,2024-04-16
5,Scott,D104,MA,6000,36,2024-04-16
6,Gen,D105,JA,21345,24,2024-04-16
7,John,D102,MY,87654,40,2024-04-16
8,Maria,D105,SA,38144,38,2024-04-16
9,Soffy,D103,IN,23456,29,2024-04-16
10,Amy,D103,CN,21345,24,2024-04-16
